In [12]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """
    # 네이버 금융 -> 시세 -> 개발자 도구 -> Networks -> Doc -> user-agent
    # 브라우저임을 알려줘야 한다
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}'
    
    response = requests.get(url, headers=headers)
    html = bs(response.text, 'lxml')
    table = html.select('table')
    temp = pd.read_html(str(table))
    return temp[0].dropna()

In [11]:
item_code = '323990'
item_name = '박셀바이오'
page_no = 1

get_day_list(item_code, page_no)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [18]:
import time
page_no  = 1
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    # 0.1초씩 쉬어간다
    time.sleep(0.1)
    # shape: (10, 7)
    # 행 10개 , 즉 데이터가 더 이상 없는 경우
    if temp.shape[0] < 10:
        break
        
item_list

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


[            날짜       종가     전일비       시가       고가       저가       거래량
 1   2021.09.03  81800.0   200.0  82500.0  83500.0  81300.0  312425.0
 2   2021.09.02  82000.0  2500.0  81000.0  83100.0  79000.0  482785.0
 3   2021.09.01  79500.0   700.0  78800.0  81500.0  78800.0  220885.0
 4   2021.08.31  78800.0   300.0  79100.0  81200.0  77500.0  365477.0
 5   2021.08.30  79100.0  2200.0  78000.0  81500.0  77000.0  594635.0
 9   2021.08.27  76900.0  2500.0  74000.0  79500.0  73100.0  442806.0
 10  2021.08.26  74400.0  1800.0  75600.0  75900.0  74000.0  192873.0
 11  2021.08.25  76200.0  2400.0  74100.0  79700.0  73000.0  569906.0
 12  2021.08.24  73800.0  3200.0  70800.0  76700.0  70700.0  422264.0
 13  2021.08.23  70600.0   300.0  70400.0  72000.0  69000.0  231102.0,
             날짜       종가     전일비       시가       고가       저가       거래량
 1   2021.08.20  70300.0  4400.0  74800.0  76000.0  70100.0  366978.0
 2   2021.08.19  74700.0  1400.0  72000.0  80900.0  71900.0  947360.0
 3   2021.08.18  73

In [21]:
df = pd.concat(item_list)

In [25]:
# 컬럼 추가
df['종목코드'] = item_code
df['종목명'] = item_name

# 순서 변경
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

# 중복 데이터 제거 (아마 없을 것)
df = df.drop_duplicatesplicates()
df.head()

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,323990,박셀바이오,2021.09.03,81800.0,200.0,82500.0,83500.0,81300.0,312425.0
2,323990,박셀바이오,2021.09.02,82000.0,2500.0,81000.0,83100.0,79000.0,482785.0
3,323990,박셀바이오,2021.09.01,79500.0,700.0,78800.0,81500.0,78800.0,220885.0
4,323990,박셀바이오,2021.08.31,78800.0,300.0,79100.0,81200.0,77500.0,365477.0
5,323990,박셀바이오,2021.08.30,79100.0,2200.0,78000.0,81500.0,77000.0,594635.0


In [26]:
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,236.000000,236.000000,236.000000,236.000000,236.000000,2.360000e+02
mean,102106.991525,5295.338983,99816.525424,105445.762712,95920.974576,1.476795e+06
std,46103.804060,8592.111640,45716.793811,50600.173482,42150.609580,2.098533e+06
min,21300.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,81725.000000,700.000000,81200.000000,83025.000000,80500.000000,3.230892e+05
50%,94350.000000,1900.000000,94600.000000,98000.000000,92000.000000,6.970240e+05
75%,113125.000000,5625.000000,113450.000000,117400.000000,110125.000000,1.676478e+06
max,269500.000000,59400.000000,279900.000000,299700.000000,232100.000000,1.451469e+07


In [29]:
# 최신 날짜
date = df.iloc[0]['날짜']
date

'2021.09.03'

In [30]:
file_name = f'{item_name}_{item_code}_{date}.csv'
file_name

'박셀바이오_323990_2021.09.03.csv'

In [31]:
df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,323990,박셀바이오,2021.09.03,81800.0,200.0,82500.0,83500.0,81300.0,312425.0
1,323990,박셀바이오,2021.09.02,82000.0,2500.0,81000.0,83100.0,79000.0,482785.0
2,323990,박셀바이오,2021.09.01,79500.0,700.0,78800.0,81500.0,78800.0,220885.0
3,323990,박셀바이오,2021.08.31,78800.0,300.0,79100.0,81200.0,77500.0,365477.0
4,323990,박셀바이오,2021.08.30,79100.0,2200.0,78000.0,81500.0,77000.0,594635.0
...,...,...,...,...,...,...,...,...,...
231,323990,박셀바이오,2020.09.28,28000.0,1050.0,27800.0,29150.0,27050.0,1628177.0
232,323990,박셀바이오,2020.09.25,26950.0,450.0,27100.0,30500.0,26000.0,4612185.0
233,323990,박셀바이오,2020.09.24,26500.0,1000.0,24450.0,31200.0,24000.0,8325838.0
234,323990,박셀바이오,2020.09.23,25500.0,4200.0,22150.0,27650.0,22050.0,8921196.0
